In [40]:
import warnings
warnings.filterwarnings("ignore")
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownTextSplitter
from sentence_transformers import SentenceTransformer, CrossEncoder
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import pandas as pd
from bert_score import score
from tqdm import tqdm


### Reading data

In [31]:
def read_doc(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()
        content = [line.strip() for line in content]
        doc = ' '.join(content)
    return doc

In [32]:
text = read_doc('./Data/outPutTxt.txt')
md = read_doc('./Data/outPutMD.md')

### Chunking

In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 500
)
md_splitter = MarkdownTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 500
)

In [34]:
doc0 = text_splitter.create_documents([text])
doc1 = md_splitter.create_documents([md])

In [35]:
text_doc = [doc.page_content for doc in doc0]
md_doc = [doc.page_content for doc in doc1]

In [36]:
print(len(text_doc))
print(len(md_doc))

1122
650


In [8]:
for chunk in enumerate(md_doc[:3]):
    print(chunk)
    print('--'*50)

(0, '# Xử lý giao dịch  (*) Thời gian giao dịch phụ thuộc vào thời gian cung cấp dịch vụ của từng ngân hàng.  Riêng rút tiền mặt tại quầy VPBank hạn mức 3.000.000.000 VND/Ngày/TKCK.  Tổ chức tài chính sẽ thông báo thêm thông tin chi tiết về các giao dịch. # Biểu Phí Dịch Vụ Ngân Hàng  |STT|Dịch vụ|Kênh giao dịch|Biểu phí| |---|---|---|---| |1|Chuyển tiền (*)|Rút tại quầy VPBank (theo giờ làm việc của VPBank)|Qua các chi nhánh VPBank tại Hà Nội: Miễn phí; Qua các chi nhánh VPBank khác: 0,03% giá trị giao dịch, tối thiểu 20.000 VND/giao dịch, tối đa 1.500.000 VND/giao dịch| |1|Chuyển tiền (*)|Chuyển tiền nhanh tới Ngân hàng liên kết|Số tiền <= 1 triệu VND: Miễn phí| |1|Chuyển tiền (*)|Chuyển tiền nhanh qua Napas|Số tiền > 1 triệu VND: 3.000 VND/giao dịch| |1|Chuyển tiền (*)|Chuyển tiền thường (Không áp dụng Ngoài giờ)|0,02% giá trị giao dịch, tối thiểu 10.000 VND/giao dịch, tối đa 600.000 VND/giao dịch| |1|Chuyển tiền (*)|Chuyển tiền tới Giấy tờ tùy thân (GTTT) (Không áp dụng Ngoài giờ)|

### Embedding

In [41]:
model_embedding = SentenceTransformer('dangvantuan/vietnamese-embedding-LongContext', trust_remote_code=True)
model_embedding

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: VietnameseModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [10]:
embeddingTxt = model_embedding.encode(text_doc)
embeddingMd = model_embedding.encode(md_doc)

In [12]:
print(embeddingTxt.shape)
print(embeddingMd.shape)

(1122, 768)
(650, 768)


### Creating Vector Database

In [ ]:
QDRANT_API_KEY = ""
QDRANT_URL = ""

In [18]:
client = QdrantClient(
    url = QDRANT_URL,
    api_key = QDRANT_API_KEY,
    https=True,
    timeout=60
)

In [19]:
collection_name = "stock_embedding"

In [25]:
client.recreate_collection(
    collection_name=collection_name,
    vectors_config = VectorParams(size = 768, distance = Distance.COSINE)
)

True

In [26]:
def upload_vector(embeddings, texts, start_id, source):
    for i, (vt, txt) in enumerate(zip(embeddings, texts)):
        client.upsert(
            collection_name=collection_name,
            points=[
                {
                    "id": start_id + i,
                    "vector": vt.tolist(),
                    "payload": {
                        "text": txt,
                        "metadata": f"Additional info vector {start_id +i }",
                        "source": source
                    }
                }
            ]
        )

In [27]:
upload_vector(embeddingTxt, text_doc, start_id=0, source="Text")

In [28]:
start_id_2 = len(embeddingTxt) + 1
start_id_2

1123

In [29]:
upload_vector(embeddingMd, md_doc, start_id=start_id_2, source="MarkDown")

In [42]:
def semantic_search(query_text, top_k):
    query_vector = model_embedding.encode(query_text)

    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_vector.tolist(),
        limit=top_k
    )

    result = []
    for item in search_result:
        result.append(
            {
                'text': item.payload['text'],
                'score': item.score,
                'metadata': item.payload.get('metadata','')
            }
        )
    return result

In [31]:
query = "giờ mở cửa"
top_result = semantic_search(query, top_k=3)

In [26]:
for i, result in enumerate(top_result, 1):
    print(f"Kết quả #{i}:")
    print(f"Text: {result['text']}")
    print(f"Score: {result['score']}")
    print(f"Metadata: {result['metadata']}")

Kết quả #1:
Text: khách hàng mở tại Công ty phải có đủ số lượng chứng khoán muốn bán.  # II. QUY ĐỊNH CHUNG VỀ GIAO DỊCH:  # 1. Nhà đầu tư chỉ được phép mở một TK GDCK tại mỗi Công ty Chứng khoán  Nhà đầu tư có thể sử dụng nhiều tài khoản khác nhau mở tại các công ty chứng khoán khác nhau để thực hiện giao dịch.  # 2. Nhà đầu tư được thực hiện các giao dịch ngược chiều (mua, bán) cùng một loại chứng khoán trong một ngày giao dịch, khi đáp ứng các điều kiện sau:  - Sử dụng một TK mở tại một CTCK để thực hiện cả lệnh mua và bán; - Các giao dịch ngược chiều (mua, bán) trên một tài khoản chỉ áp dụng trong phiên khớp lệnh liên tục, không áp dụng trong phiên khớp lệnh định kỳ.  # 3. Nhà đầu tư không được phép: # III. QUY ĐỊNH GIAO DỊCH TẠI CÁC SỞ GIAO DỊCH CHỨNG KHOÁN:  # A. Chứng Khoán Niêm Yết tại Sở Giao Dịch Chứng Khoán TP.HCM (HSX)  # 1. Thời gian giao dịch:  từ thứ Hai đến thứ Sáu hàng tuần, trừ các ngày nghỉ theo quy định của Bộ luật Lao động và những ngày nghỉ giao dịch theo quy định

### Setting LLM

In [44]:
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    https=True
)

In [45]:
bi_encoder = SentenceTransformer('dangvantuan/vietnamese-embedding-LongContext', trust_remote_code=True)
cross_encoder = CrossEncoder('itdainb/PhoRanker')

### Local LLM

In [8]:
from openai import OpenAI

llm_client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [53]:
def semantic_search_rerank(query_text, top_k=15, rerank_top_k=5):
    query_vector = bi_encoder.encode(query_text)

    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector.tolist(),
        limit=top_k
    )

    rerank_pairs = [(query_text, item.payload['text']) for item in search_result]
    rerank_score = cross_encoder.predict(rerank_pairs)

    for i, item in enumerate(search_result):
        item.score = float(rerank_score[i])
    
    rerank_results = sorted(search_result, key=lambda x: x.score, reverse=True)[:rerank_top_k]

    return [item.payload['text'] for item in rerank_results]

In [10]:
def generate_answer(query, context):
    prompt = f"""
        Bạn là một người sử dụng tiếng Việt thành thạo.
        Bạn cũng là một chuyên gia tư vấn về lĩnh vực chứng khoán.
        Dựa vào ngữ cảnh dưới đây, hãy trả lời câu hỏi một cách ngắn gọn và chính xác nhất bằng Tiếng Việt.
        Nếu không biết câu trả lời, hãy lịch sử bảo rằng không biết và không bịa đặt nội dung.
        Ngữ cảnh: {context}
        Câu hỏi: {query}
        Câu trả lời (bằng Tiếng Việt):"""
    
    completion = llm_client.chat.completions.create(
        model = "RichardErkhov/vilm_-_vietcuna-7b-v3-gguf/vietcuna-7b-v3.Q5_K_S.gguf",
        messages=[
            {
                "role": "system",
                "content": "Bạn là trợ lý AI sử dụng tiếng Việt thành thạo."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.8,
        max_tokens=300
    )

    return completion.choices[0].message.content

In [84]:
def generate_answer(query, context):
    prompt = f"""
    Bạn là một người sử dụng tiếng việt thành thạo.
    Dựa vào ngữ cảnh sau đây, hãy trả lời câu hỏi một cách ngắn gọn và chính xác bằng Tiếng Việt.
    Nếu không biết câu trả lời hãy lịch sự bảo không biết, không bịa đặt nội dung:

    Ngữ cảnh: {context}

    Câu hỏi: {query}

    Trả lời:"""

    completion = llm_client.chat.completions.create(
        model="nguyenviet/PhoGPT-7B5-Instruct-GGUF/PhoGPT-7B5-Instruct-q5_k_m.gguf",
        messages=[
            {"role": "system", "content": "Bạn là trợ lý AI sử dụng tiếng Việt thành thạo."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.6,
        max_tokens=300,
        # top_p=0.9,
        # frequency_penalty=0.3,
        # presence_penalty=0.3
    )

    return completion.choices[0].message.content

### API LLM

In [ ]:
XAI_API_KEY = ""

In [48]:
from openai import OpenAI

In [49]:
client = OpenAI(
  api_key=XAI_API_KEY,
  base_url="https://api.x.ai/v1",
)

In [50]:
def generate_answer(query, context):
    prompt = f"""
            Bạn là một người sử dụng tiếng Việt thành thạo.
            Bạn cũng là một chuyên gia tư vấn về lĩnh vực chứng khoán.
            Dựa vào ngữ cảnh dưới đây, hãy trả lời câu hỏi một cách ngắn gọn và chính xác nhất bằng Tiếng Việt.
            Nếu không biết câu trả lời, hãy lịch sử bảo rằng không biết và không bịa đặt nội dung.
            Luôn trả lời bằng Tiếng Việt, không sử dụng bất cứ ngôn ngữ nào khác:

        Ngữ cảnh: {context}

        Câu hỏi: {query}

        Câu trả lời (bằng Tiếng Việt):"""

    completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Bạn là trợ lý AI sử dụng tiếng Việt thành thạo. Trả lời ngắn gọn, chính xác, không thêm thông tin ngoài lề"
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="grok-2-1212",
    )

    return completion.choices[0].message.content

In [ ]:
GROQ_API_KEY = ""

In [11]:
from groq import Groq

client = Groq(
    api_key=GROQ_API_KEY
)

In [12]:
def generate_answer(query, context):
    prompt = f"""
            Bạn là một người sử dụng tiếng Việt thành thạo.
            Bạn cũng là một chuyên gia tư vấn về lĩnh vực chứng khoán.
            Dựa vào ngữ cảnh dưới đây, hãy trả lời câu hỏi một cách ngắn gọn và chính xác nhất bằng Tiếng Việt.
            Nếu không biết câu trả lời, hãy lịch sử bảo rằng không biết và không bịa đặt nội dung.
            Luôn trả lời bằng Tiếng Việt, không sử dụng bất cứ ngôn ngữ nào khác:

        Ngữ cảnh: {context}

        Câu hỏi: {query}

        Câu trả lời (bằng Tiếng Việt):"""

    completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Bạn là trợ lý AI sử dụng tiếng Việt thành thạo. Trả lời ngắn gọn, chính xác, không thêm thông tin ngoài lề"
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    return completion.choices[0].message.content

### Generating from LLM

In [51]:
def qa_system(query):
    relevant_passages = semantic_search_rerank(query)
    context = " ".join(relevant_passages)
    answer = generate_answer(query, context)
    return answer

In [52]:
user_query = "Tôi có thể mở nhiều tài khoản chứng khoán tại VPS không?"
response = qa_system(user_query)

print(f"Câu hỏi: {user_query}")
print(f"Trả lời: {response}")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Câu hỏi: Tôi có thể mở nhiều tài khoản chứng khoán tại VPS không?
Trả lời: Theo quy định của cơ quan quản lý, mỗi khách hàng chỉ có thể sử dụng chứng minh nhân dân/căn cước công dân còn hiệu lực của cá nhân đó để mở 01 tài khoản giao dịch chứng khoán tại một công ty chứng khoán. Quý khách có thể mở tài khoản giao dịch chứng khoán tại các công ty chứng khoán khác nhau.


### Testing LLM response

#### API LLM

In [30]:
data = pd.read_csv('./Data/sm_test.csv')
data.tail()

,Questions,Answers
5,Tôi có thể mở tài khoản giao dịch tại VPS qua ...,"""Để mở tài khoản giao dịch tại VPS, Quý khách ..."
6,Tôi có thể mở tài khoản tại VPS vào thời điểm ...,"""Hệ thống mở tài khoản trực tuyến của VPS hoạt..."
7,"""Tôi quét QR chuyển tiền, người nhận chưa nhận...",• Quý khách vui lòng liên hệ tới Tổng đài 1900...
8,Nạp tiền vào tài khoản chứng khoán từ tài khoả...,Hạn mức nạp tiền vào tài khoản chứng khoán (TK...
9,"""Tôi thấy khi tạo mã QR nhận tiền từ TKCK luôn...","""Quý khách có thể đổi sang các loại tài khoản ..."


In [11]:
def evaluate_response(csv_file_path):
    df = pd.read_csv(csv_file_path)

    questions = df['Questions'].tolist()
    reference_answers = df['Answers'].tolist()

    llm_answers = []
    for quest in tqdm(questions, desc="Generating LLM answers"):
        each_llm_answer = qa_system(quest)
        llm_answers.append(each_llm_answer)

    P, R, F1 = score(llm_answers, reference_answers, lang="vi", verbose=True)

    avg_precision = P.mean().item()
    avg_recall = R.mean().item()
    avg_f1 = F1.mean().item()

    return {
        'avg_precision': avg_precision,
        'avg_recall': avg_recall,
        'avg_f1': avg_f1,
        'individual_scores': list(zip(questions, reference_answers, llm_answers, F1.tolist()))
    }

In [90]:
csv_file_path = './Data/test_1_100.csv'
result = evaluate_response(csv_file_path=csv_file_path)

Generating LLM answers:   0%|          | 0/99 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are n

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 18.41 seconds, 5.38 sentences/sec


In [91]:
print(f"Average Precision: {result['avg_precision']:.4f}")
print(f"Average Recall: {result['avg_recall']:.4f}")
print(f"Average F1 Score: {result['avg_f1']:.4f}")


Average Precision: 0.8514
Average Recall: 0.8194
Average F1 Score: 0.8341


In [49]:
print("\nMột số ví dụ cụ thể:")
for question, ref_answer, llm_answer, f1_score in result['individual_scores'][:5]:
    print(f"Question: {question}")
    print('--'*30)
    print(f"Reference Answer: {ref_answer}")
    print('--'*30)
    print(f"LLM Answer: {llm_answer}")
    print('--'*30)
    print(f"F1 Score: {f1_score:.4f}\n")
    print('##'*50)


Một số ví dụ cụ thể:
Question: Tôi có thể mở nhiều tài khoản chứng khoán tại VPS không?
------------------------------------------------------------
Reference Answer: "Theo quy định của cơ quan quản lý, mỗi khách hàng chỉ có thể sử dụng chứng minh nhân dân/căn cước công dân còn hiệu lực của cá nhân đó để mở 01 tài khoản giao dịch chứng khoán tại một công ty chứng khoán.Quý khách có thể mở tài khoản giao dịch chứng khoán tại các công ty chứng khoán khác nhau."
------------------------------------------------------------
LLM Answer: Theo quy định của cơ quan quản lý, mỗi khách hàng chỉ có thể sử dụng chứng minh nhân dân/căn cước công dân còn hiệu lực của cá nhân đó để mở 01 tài khoản giao dịch chứng khoán tại một công ty chứng khoán. Do đó, bạn chỉ có thể mở 01 tài khoản chứng khoán tại VPS.
------------------------------------------------------------
F1 Score: 0.9275

####################################################################################################
Question: 

In [54]:
import gradio as gr

In [ ]:
iface = gr.Interface(
 fn=qa_system,
 inputs=[gr.Textbox(label="Question")],  # Pass input as a list
 outputs=[gr.Textbox(label="Generated Response")],  # Pass output as a list
 title="Stock Chatbot support for VPS",
#  description="Enter a question and get a generated response based on the retrieved text.",
)

iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

#### Local LLM

In [14]:
csv_file_path = './Data/test_1_100.csv'
result = evaluate_response(csv_file_path=csv_file_path)

Generating LLM answers:   0%|          | 0/99 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are n

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 19.21 seconds, 5.15 sentences/sec


In [15]:
print(f"Average Precision: {result['avg_precision']:.4f}")
print(f"Average Recall: {result['avg_recall']:.4f}")
print(f"Average F1 Score: {result['avg_f1']:.4f}")

Average Precision: 0.7180
Average Recall: 0.7061
Average F1 Score: 0.7092


In [16]:
print("\nMột số ví dụ cụ thể:")
for question, ref_answer, llm_answer, f1_score in result['individual_scores'][:5]:
    print(f"Question: {question}")
    print('--'*30)
    print(f"Reference Answer: {ref_answer}")
    print('--'*30)
    print(f"LLM Answer: {llm_answer}")
    print('--'*30)
    print(f"F1 Score: {f1_score:.4f}\n")
    print('##'*50)


Một số ví dụ cụ thể:
Question: Tôi có thể mở nhiều tài khoản chứng khoán tại VPS không?
------------------------------------------------------------
Reference Answer: "Theo quy định của cơ quan quản lý, mỗi khách hàng chỉ có thể sử dụng chứng minh nhân dân/căn cước công dân còn hiệu lực của cá nhân đó để mở 01 tài khoản giao dịch chứng khoán tại một công ty chứng khoán.Quý khách có thể mở tài khoản giao dịch chứng khoán tại các công ty chứng khoán khác nhau."
------------------------------------------------------------
LLM Answer: Bạn có thể mở nhiều tài khoản chứng khoán tại VPS.
------------------------------------------------------------
F1 Score: 0.7535

####################################################################################################
Question: Tôi muốn ủy quyền cho người khác để thực hiện mở tài khoản chứng khoán thì cần thủ tục gì?
------------------------------------------------------------
Reference Answer: "Theo quy định của VPS, Quý khách không đượ

#### PhoGPT Humaneval

In [99]:
print("\nMột số ví dụ cụ thể:")
for question, ref_answer, llm_answer, f1_score in result['individual_scores'][:5]:
    print(f"Question: {question}")
    print('--'*30)
    print(f"Reference Answer: {ref_answer}")
    print('--'*30)
    print(f"LLM Answer: {llm_answer}")
    print('--'*30)
    print(f"F1 Score: {f1_score:.4f}\n")
    print('##'*50)


Một số ví dụ cụ thể:
Question: Tôi có thể mở nhiều tài khoản chứng khoán tại VPS không?
------------------------------------------------------------
Reference Answer: "Theo quy định của cơ quan quản lý, mỗi khách hàng chỉ có thể sử dụng chứng minh nhân dân/căn cước công dân còn hiệu lực của cá nhân đó để mở 01 tài khoản giao dịch chứng khoán tại một công ty chứng khoán.Quý khách có thể mở tài khoản giao dịch chứng khoán tại các công ty chứng khoán khác nhau."
------------------------------------------------------------
LLM Answer: Theo quy định của VPS, mỗi khách hàng chỉ được sử dụng chứng minh nhân dân/Căn cước công dân có hạn sử dụng để mở tài khoản giao dịch. Vì vậy, nếu muốn mở nhiều tài khoản giao dịch chứng khoán, quý khách cần liên hệ với chi nhánh/phòng giao dịch gần nhất của VPS để thực hiện các thủ tục liên quan đến việc mở tài khoản mới.
------------------------------------------------------------
F1 Score: 0.8058

##################################################

#### LLM test without RAG